In [1]:
import sys

sys.path.append("../../")

from models.CNN import CNN_v2 as cnn
from stats import evaluate_perf_utils as eval
from utils import load_csv
import numpy as np

import plotly.graph_objects as go
import numpy as np


2024-12-12 10:38:15.303735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733996295.438363   43970 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733996295.480363   43970 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 10:38:15.805712: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import os
print(os.getcwd())

model = cnn.CNN_v2()
all_data = load_csv.load_pandas()

_, _, test_set = load_csv.split_data(all_data)

model.load_model("./cnn.keras")

model.load_data(test_set=test_set)

# model.save_model("cnn.keras")
y_predict, _ = model.evaluate()
# print("y_predict shape: ", y_predict.shape)
# print("y_predict all elements: ", y_predict[:200])

metrics = eval.evaluate_performance(model.y_test, y_predict)

tn = metrics["tn"]
fp = metrics["fp"]
fn = metrics["fn"]
tp = metrics["tp"]

# Confusion matrix
cm = np.array([[tn, fp], [fn, tp]])

total = cm.sum()
cm_percentage = cm / total * 100

# Create a heatmap for the confusion matrix with percentages
fig = go.Figure(
    data=go.Heatmap(
        z=cm_percentage,
        x=["Pred: 0", "Pred: 1"],
        y=["True: 0", "True: 1"],
        colorscale="Peach",  # A more pleasant color palette
        colorbar=dict(title="Percentage", tickvals=[0, 100]),
        showscale=True,
    )
)

# Add titles, labels, and precision, recall, F1 score as annotation
fig.update_layout(
    title="Confusion Matrix (Percentage)",
    xaxis_title="Predicted Labels",
    yaxis_title="True Labels",
    width=500,  # Set the width of the plot
    height=400,  # Set the height of the plot
)

# Add text annotations to show percentage values in each cell
for i in range(cm_percentage.shape[0]):
    for j in range(cm_percentage.shape[1]):
        fig.add_annotation(
            x=j,
            y=i,
            text=f"{cm_percentage[i, j]:.2f}%",
            showarrow=False,
            font=dict(size=14, color="black"),
            align="center",
        )

# Show the plot
fig.show()

/home/thibaud/majority-vote-outlier-detection/models/CNN


/home/thibaud/majority-vote-outlier-detection/models/CNN/../../utils/load_csv.py:20: DtypeWarning: Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(data_path, delimiter=";").iloc[:, -3:]
2024-12-12 10:39:28.336785: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Existing model found. Only loading test data.


/home/thibaud/majority-vote-outlier-detection/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
Test Accuracy: 95.093
